In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
            SparkSession
                .builder
                .appName("DeltaLakeApp")
    
                .master("local[*]")    
                .config("spark.dynamicAllocation.enabled", "false")     
    
    
                # Add package for Delta Lake
                .config("spark.jars.packages", "io.delta:delta-core_2.12:3.2.5")
    
    
                # Add settings to use Delta Lake with Spark session
                .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    
                .config("spark.sql.catalog.spark_catalog", 
                        "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    
                .getOrCreate()
        )

sc = spark.sparkContext

spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# Create schema for Yellow Taxi data
yellowTaxiSchema = (
                        StructType
                        ([ 
                            StructField("VendorId"               , IntegerType()   , True),
                            StructField("PickupTime"             , TimestampType() , True),
                            StructField("DropTime"               , TimestampType() , True),
                            StructField("PassengerCount"         , DoubleType()    , True),
                            StructField("TripDistance"           , DoubleType()    , True),
                            StructField("RateCodeId"             , DoubleType()    , True),
                            StructField("StoreAndFwdFlag"        , StringType()    , True),
                            StructField("PickupLocationId"       , IntegerType()   , True),
                            StructField("DropLocationId"         , IntegerType()   , True),
                            StructField("PaymentType"            , IntegerType()   , True),
                            StructField("FareAmount"             , DoubleType()    , True),
                            StructField("Extra"                  , DoubleType()    , True),
                            StructField("MtaTax"                 , DoubleType()    , True),
                            StructField("TipAmount"              , DoubleType()    , True),
                            StructField("TollsAmount"            , DoubleType()    , True),
                            StructField("ImprovementSurcharge"   , DoubleType()    , True),
                            StructField("TotalAmount"            , DoubleType()    , True),
                            StructField("CongestionSurcharge"    , DoubleType()    , True),
                            StructField("AirportFee"             , DoubleType()    , True)
                        ])
                   )


# Read Yellow Taxis file
yellowTaxiDF = (
                  spark
                    .read
                    .option("header", "true")    
                    .schema(yellowTaxiSchema)    
                    .csv("/FileStore/tables/YellowTaxis_202210.csv")
               )


# Print schema of DataFrame
yellowTaxiDF.printSchema()

root
 |-- VendorId: integer (nullable = true)
 |-- PickupTime: timestamp (nullable = true)
 |-- DropTime: timestamp (nullable = true)
 |-- PassengerCount: double (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- RateCodeId: double (nullable = true)
 |-- StoreAndFwdFlag: string (nullable = true)
 |-- PickupLocationId: integer (nullable = true)
 |-- DropLocationId: integer (nullable = true)
 |-- PaymentType: integer (nullable = true)
 |-- FareAmount: double (nullable = true)
 |-- Extra: double (nullable = true)
 |-- MtaTax: double (nullable = true)
 |-- TipAmount: double (nullable = true)
 |-- TollsAmount: double (nullable = true)
 |-- ImprovementSurcharge: double (nullable = true)
 |-- TotalAmount: double (nullable = true)
 |-- CongestionSurcharge: double (nullable = true)
 |-- AirportFee: double (nullable = true)



### Create database in metastore

In [0]:
spark.sql("""

CREATE DATABASE IF NOT EXISTS TaxisDB

""")

Out[5]: DataFrame[]

### Parquet format: Save DataFrame as a Table

In [0]:
(
    yellowTaxiDF
            .write    
            .mode("overwrite")
    
            .partitionBy("VendorId")
    
            .format("parquet")
    
            .option("path", "/FileStore/tables/Output/YellowTaxis.parquet")
    
            .saveAsTable("TaxisDB.YellowTaxisParquet")
)

In [0]:
display(dbutils.fs.ls("/FileStore/tables/"))


path,name,size,modificationTime
dbfs:/FileStore/tables/2010_12_01-1.csv,2010_12_01-1.csv,275001,1729878680000
dbfs:/FileStore/tables/2010_12_01-2.csv,2010_12_01-2.csv,275001,1729932035000
dbfs:/FileStore/tables/2010_12_01.csv,2010_12_01.csv,275001,1729878505000
dbfs:/FileStore/tables/Cabs.csv,Cabs.csv,1365691,1730487220000
dbfs:/FileStore/tables/CacheTest_EnabledAndCached.csv/,CacheTest_EnabledAndCached.csv/,0,0
dbfs:/FileStore/tables/CacheTest_EnabledFirstTime.csv/,CacheTest_EnabledFirstTime.csv/,0,0
dbfs:/FileStore/tables/CacheTest_WithoutEnabling.csv/,CacheTest_WithoutEnabling.csv/,0,0
dbfs:/FileStore/tables/Drivers.csv,Drivers.csv,11061205,1730487592000
dbfs:/FileStore/tables/GreenTaxis_202210.csv,GreenTaxis_202210.csv,8747800,1730487440000
dbfs:/FileStore/tables/Output/,Output/,0,0


### Delta format: Save DataFrame as Table

In [0]:
# If you have already run this command, and want to start from beginning,
# delete folder from file system first

from delta import *

(
    yellowTaxiDF
            .write    
            .mode("overwrite")
    
            .partitionBy("VendorId")
    
            .format("delta")
    
            .option("path", "/FileStore/tables/Output/YellowTaxis.delta")
    
            .saveAsTable("TaxisDB.YellowTaxis")
)

In [0]:
display(dbutils.fs.ls("/FileStore/tables/"))


path,name,size,modificationTime
dbfs:/FileStore/tables/2010_12_01-1.csv,2010_12_01-1.csv,275001,1729878680000
dbfs:/FileStore/tables/2010_12_01-2.csv,2010_12_01-2.csv,275001,1729932035000
dbfs:/FileStore/tables/2010_12_01.csv,2010_12_01.csv,275001,1729878505000
dbfs:/FileStore/tables/Cabs.csv,Cabs.csv,1365691,1730487220000
dbfs:/FileStore/tables/CacheTest_EnabledAndCached.csv/,CacheTest_EnabledAndCached.csv/,0,0
dbfs:/FileStore/tables/CacheTest_EnabledFirstTime.csv/,CacheTest_EnabledFirstTime.csv/,0,0
dbfs:/FileStore/tables/CacheTest_WithoutEnabling.csv/,CacheTest_WithoutEnabling.csv/,0,0
dbfs:/FileStore/tables/Drivers.csv,Drivers.csv,11061205,1730487592000
dbfs:/FileStore/tables/GreenTaxis_202210.csv,GreenTaxis_202210.csv,8747800,1730487440000
dbfs:/FileStore/tables/Output/,Output/,0,0


In [0]:
spark.sql("""

SELECT COUNT(*)
FROM TaxisDB.YellowTaxis

""").show()

+--------+
|count(1)|
+--------+
| 3675412|
+--------+



### Audit History of Delta Table

This shows transaction log of Delta Table

In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+---------------------------------+----------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+----------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId          |userName          |operation                        |operationParameters                                                                     |job |notebook         |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                                      |userMetadata|engineInfo                         |
+-------+-------------------+----------------+------------------+---------------------------------+----------------------------------------------------------------------------------------+----+-------------

### Overwrite data in Delta table

In [0]:
# Overwrite data by re-running the command

(
    yellowTaxiDF
            .write    
            .mode("overwrite")
    
            .partitionBy("VendorId")
    
            .format("delta")
    
            .option("path", "/FileStore/tables/Output/YellowTaxis.delta")
    
            .saveAsTable("TaxisDB.YellowTaxis")
)

In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+---------------------------------+----------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+----------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId          |userName          |operation                        |operationParameters                                                                     |job |notebook         |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                                      |userMetadata|engineInfo                         |
+-------+-------------------+----------------+------------------+---------------------------------+----------------------------------------------------------------------------------------+----+-------------

In [0]:
spark.sql("""

DROP TABLE TaxisDB.YellowTaxis

""")

Out[18]: DataFrame[]

In [0]:
dbutils.fs.rm("/FileStore/tables/Output/", recurse=True)



Out[28]: False

In [0]:
display(dbutils.fs.ls("/FileStore/tables/"))

[FileInfo(path='dbfs:/FileStore/tables/2010_12_01-1.csv', name='2010_12_01-1.csv', size=275001, modificationTime=1729878680000),
 FileInfo(path='dbfs:/FileStore/tables/2010_12_01-2.csv', name='2010_12_01-2.csv', size=275001, modificationTime=1729932035000),
 FileInfo(path='dbfs:/FileStore/tables/2010_12_01.csv', name='2010_12_01.csv', size=275001, modificationTime=1729878505000),
 FileInfo(path='dbfs:/FileStore/tables/Cabs.csv', name='Cabs.csv', size=1365691, modificationTime=1730487220000),
 FileInfo(path='dbfs:/FileStore/tables/CacheTest_EnabledAndCached.csv/', name='CacheTest_EnabledAndCached.csv/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/CacheTest_EnabledFirstTime.csv/', name='CacheTest_EnabledFirstTime.csv/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/CacheTest_WithoutEnabling.csv/', name='CacheTest_WithoutEnabling.csv/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/Drivers.csv', name='Drivers.csv', size=1

### Create Table Definition

Create table using DDL command, and later add the data

In [0]:
spark.sql("""

CREATE TABLE TaxisDB.YellowTaxis
(
    VendorId                INT               COMMENT 'Vendor providing the ride',
    
    PickupTime              TIMESTAMP,
    DropTime                TIMESTAMP,
    
    PickupLocationId        INT               NOT NULL,
    DropLocationId          INT,
    
    PassengerCount          DOUBLE,
    TripDistance            DOUBLE,
    
    RateCodeId              DOUBLE,
    StoreAndFwdFlag         STRING,
    PaymentType             INT,
    
    FareAmount              DOUBLE,
    Extra                   DOUBLE,
    MtaTax                  DOUBLE,
    TipAmount               DOUBLE,
    TollsAmount             DOUBLE,
    ImprovementSurcharge    DOUBLE,
    TotalAmount             DOUBLE,
    CongestionSurcharge     DOUBLE,
    AirportFee              DOUBLE
)

USING DELTA                  -- default is Parquet

LOCATION "/FileStore/tables/Output/YellowTaxis.delta/"

PARTITIONED BY (VendorId)    -- optional

COMMENT 'This table stores ride information for Yellow Taxis'

""")

Out[30]: DataFrame[]

In [0]:
spark.sql("""

DESCRIBE TABLE EXTENDED TaxisDB.YellowTaxis

""").show(50, truncate=False)

+----------------------------+---------------------------------------------------+-------------------------+
|col_name                    |data_type                                          |comment                  |
+----------------------------+---------------------------------------------------+-------------------------+
|VendorId                    |int                                                |Vendor providing the ride|
|PickupTime                  |timestamp                                          |null                     |
|DropTime                    |timestamp                                          |null                     |
|PickupLocationId            |int                                                |null                     |
|DropLocationId              |int                                                |null                     |
|PassengerCount              |double                                             |null                     |
|TripDistance      

### Options to Add Data to Delta Table

#### Option 1: Insert command

Use typical SQL Insert command to add data to table

In [0]:
spark.sql("""

INSERT INTO TaxisDB.YellowTaxis

-- (VendorId, PickupTime, DropTime, PickupLocationId, DropLocationId, PassengerCount, TripDistance, RateCodeId, StoreAndFwdFlag, PaymentType, FareAmount, Extra, MtaTax, TipAmount, TollsAmount, ImprovementSurcharge, TotalAmount, CongestionSurcharge, AirportFee)

VALUES (3, '2022-12-01T00:00:00.000Z', '2022-12-01T00:15:34.000Z', 170, 140, 1.0, 2.9, 1.0, '1', 1, 13.0, 0.5, 0.5, 1.0, 0.0, 0.3, 15.3, 0.0, 0.0)

""")

Out[32]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("""

SELECT * FROM TaxisDB.YellowTaxis

""").show(truncate=False)

+--------+-------------------+-------------------+----------------+--------------+--------------+------------+----------+---------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|VendorId|PickupTime         |DropTime           |PickupLocationId|DropLocationId|PassengerCount|TripDistance|RateCodeId|StoreAndFwdFlag|PaymentType|FareAmount|Extra|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|TotalAmount|CongestionSurcharge|AirportFee|
+--------+-------------------+-------------------+----------------+--------------+--------------+------------+----------+---------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|3       |2022-12-01 00:00:00|2022-12-01 00:15:34|170             |140           |1.0           |2.9         |1.0       |1              |1          |13.0      |0.5  |0.5   |1.0      |0.0        |0.3                 |15.3

In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+------------+---------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId          |userName          |operation   |operationParameters                                                                                                                    |job |notebook         |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-------------------+----------------+------------------+------------+--------------------------------------------------------------------------------------------------

#### Option 2: Append a DataFrame

Read data as a DataFrame and append to Delta Table using PySpark

In [0]:
# Extract new records from Storage/Data Lake

yellowTaxisAppendDF = (
                          spark
                            .read
                            .option("header", "true")
                            .schema(yellowTaxiSchema)
                            .csv("/FileStore/tables/YellowTaxis_append.csv")
                      )

yellowTaxisAppendDF.show()

+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|VendorId|         PickupTime|           DropTime|PassengerCount|TripDistance|RateCodeId|StoreAndFwdFlag|PickupLocationId|DropLocationId|PaymentType|FareAmount|Extra|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|TotalAmount|CongestionSurcharge|AirportFee|
+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|       3|2022-10-01 00:03:41|2022-10-01 00:18:39|           1.0|         1.7|       1.0|              N|             249|           107|          1|       9.5|  3.0|   0.5|     2.65|        0.0|                 0.3|    

In [0]:
# Append to data lake in delta format

(
    yellowTaxisAppendDF
        .write
  
        .mode("append")
  
        .partitionBy("VendorId")  
        .format("delta")           
        .save("/FileStore/tables/Output/YellowTaxis.delta")
)

In [0]:
spark.sql("""

SELECT * 
FROM TaxisDB.YellowTaxis

""").show()

+--------+-------------------+-------------------+----------------+--------------+--------------+------------+----------+---------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|VendorId|         PickupTime|           DropTime|PickupLocationId|DropLocationId|PassengerCount|TripDistance|RateCodeId|StoreAndFwdFlag|PaymentType|FareAmount|Extra|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|TotalAmount|CongestionSurcharge|AirportFee|
+--------+-------------------+-------------------+----------------+--------------+--------------+------------+----------+---------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|       3|2022-10-01 00:03:41|2022-10-01 00:18:39|             249|           107|           1.0|         1.7|       1.0|              N|          1|       9.5|  3.0|   0.5|     2.65|        0.0|                 0.3|    

In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+------------+---------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId          |userName          |operation   |operationParameters                                                                                                                    |job |notebook         |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-------------------+----------------+------------------+------------+--------------------------------------------------------------------------------------------------

In [0]:
# Append to data lake in delta format
(
    yellowTaxiDF
        .write
  
        .mode("append")
  
        .partitionBy("VendorId")  
        .format("delta")
        .save("/FileStore/tables/Output/YellowTaxis.delta")
)

In [0]:
spark.sql("""

SELECT COUNT(1)
FROM TaxisDB.YellowTaxis

""").show()

+--------+
|count(1)|
+--------+
| 3675416|
+--------+



### Check files holding the data

In [0]:
spark.sql("""

SELECT INPUT_FILE_NAME()
     
     , VendorId
     , PickupLocationId
     , PassengerCount
     
FROM TaxisDB.YellowTaxis

WHERE VendorId = 3

""").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------+--------+----------------+--------------+
|input_file_name()                                                                                                             |VendorId|PickupLocationId|PassengerCount|
+------------------------------------------------------------------------------------------------------------------------------+--------+----------------+--------------+
|dbfs:/FileStore/tables/Output/YellowTaxis.delta/VendorId=3/part-00000-7f73dcb3-2345-4812-9938-5f95b3af4d03.c000.snappy.parquet|3       |249             |1.0           |
|dbfs:/FileStore/tables/Output/YellowTaxis.delta/VendorId=3/part-00000-7f73dcb3-2345-4812-9938-5f95b3af4d03.c000.snappy.parquet|3       |151             |2.0           |
|dbfs:/FileStore/tables/Output/YellowTaxis.delta/VendorId=3/part-00000-7f73dcb3-2345-4812-9938-5f95b3af4d03.c000.snappy.parquet|3       |238          

### UPDATE command

In [0]:
# Check passenger count

spark.sql("""

SELECT VendorId
     , PickupLocationId
     , PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 249

""").show()

+--------+----------------+--------------+
|VendorId|PickupLocationId|PassengerCount|
+--------+----------------+--------------+
|       3|             249|           1.0|
+--------+----------------+--------------+



In [0]:
# Run update statement to change passenger count

spark.sql("""

UPDATE TaxisDB.YellowTaxis

SET PassengerCount = 2

WHERE VendorId = 3
    AND PickupLocationId = 249

""").show()

+-----------------+
|num_affected_rows|
+-----------------+
|                1|
+-----------------+



In [0]:
# Check passenger count after update

spark.sql("""

SELECT VendorId
     , PickupLocationId
     , PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 249

""").show()

+--------+----------------+--------------+
|VendorId|PickupLocationId|PassengerCount|
+--------+----------------+--------------+
|       3|             249|           2.0|
+--------+----------------+--------------+



In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+------------+---------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId          |userName          |operation   |operationParameters                                                                                                                    |job |notebook         |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                          

### DELETE command

In [0]:
# Check if record exist

spark.sql("""

SELECT VendorId
     , PickupLocationId
     , PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 151

""").show()

+--------+----------------+--------------+
|VendorId|PickupLocationId|PassengerCount|
+--------+----------------+--------------+
|       3|             151|           2.0|
+--------+----------------+--------------+



In [0]:
# Delete the record

spark.sql("""

DELETE FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 151

""").show()

+-----------------+
|num_affected_rows|
+-----------------+
|                1|
+-----------------+



In [0]:
# Check if record exist after delete operation

spark.sql("""

SELECT VendorId
     , PickupLocationId
     , PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 151

""").show()

+--------+----------------+--------------+
|VendorId|PickupLocationId|PassengerCount|
+--------+----------------+--------------+
+--------+----------------+--------------+



In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+------------+---------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId          |userName          |operation   |operationParameters                                                                                                                    |job |notebook         |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                          

### MERGE command

In [0]:
# Extract changed records from Storage/Data Lake

yellowTaxiChangesDF = (
                          spark
                            .read
                            .option("header", "true")
                            .schema(yellowTaxiSchema)
                            .csv("/FileStore/tables/YellowTaxis_changes.csv")
                      )

yellowTaxiChangesDF.show()

+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|VendorId|         PickupTime|           DropTime|PassengerCount|TripDistance|RateCodeId|StoreAndFwdFlag|PickupLocationId|DropLocationId|PaymentType|FareAmount|Extra|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|TotalAmount|CongestionSurcharge|AirportFee|
+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|       3|2022-10-01 00:03:41|2022-10-01 00:18:39|           1.0|         1.7|       1.0|              N|             249|           107|          3|       9.5|  3.0|   0.5|     2.65|        0.0|                 0.3|    

In [0]:
yellowTaxiChangesDF.createOrReplaceTempView("YellowTaxiChanges")

In [0]:
spark.sql("""

SELECT *
FROM YellowTaxiChanges

""").show()

+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|VendorId|         PickupTime|           DropTime|PassengerCount|TripDistance|RateCodeId|StoreAndFwdFlag|PickupLocationId|DropLocationId|PaymentType|FareAmount|Extra|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|TotalAmount|CongestionSurcharge|AirportFee|
+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|       3|2022-10-01 00:03:41|2022-10-01 00:18:39|           1.0|         1.7|       1.0|              N|             249|           107|          3|       9.5|  3.0|   0.5|     2.65|        0.0|                 0.3|    

In [0]:
spark.sql("""

MERGE INTO TaxisDB.YellowTaxis tgt

    USING YellowTaxiChanges    src

        ON    tgt.VendorId          =  src.VendorId
          AND tgt.PickupLocationId  =  src.PickupLocationId
  
-- Update row if join conditions match
WHEN MATCHED
      
      THEN  
          UPDATE SET    tgt.PaymentType = src.PaymentType   
          
                                                      -- Use 'UPDATE SET *' to update all columns

-- Insert row if row is not present in target table
WHEN NOT MATCHED 
      AND PickupTime >= '2022-03-01'

      THEN 
          INSERT (VendorId, PickupTime, DropTime, PickupLocationId, DropLocationId, PassengerCount, TripDistance, 
                  RateCodeId, StoreAndFwdFlag, PaymentType, FareAmount, Extra, MtaTax, TipAmount, TollsAmount, 
                  ImprovementSurcharge, TotalAmount, CongestionSurcharge, AirportFee)
          
          VALUES (VendorId, PickupTime, DropTime, PickupLocationId, DropLocationId, PassengerCount, TripDistance,
                  RateCodeId, StoreAndFwdFlag, PaymentType, FareAmount, Extra, MtaTax, TipAmount, TollsAmount, 
                  ImprovementSurcharge, TotalAmount, CongestionSurcharge, AirportFee)
                                                                         


""").show()

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                4|               3|               0|                1|
+-----------------+----------------+----------------+-----------------+



In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Constraints on Delta Table

#### NOT NULL constraint

Drop the constraint here which was previously created

<i>- Define NOT NULL constraint on column to avoid insertion of NULL values </i> <br/>

In [0]:
spark.sql("""

ALTER TABLE TaxisDB.YellowTaxis

      CHANGE COLUMN PickupLocationId DROP NOT NULL

""").show()


# ALTER TABLE TaxisDB.YellowTaxis
#      CHANGE COLUMN PickupLocationId SET NOT NULL

++
||
++
++



In [0]:
spark.sql("""

ALTER TABLE TaxisDB.YellowTaxis

      CHANGE COLUMN PickupLocationId SET NOT NULL

""").show()


# ALTER TABLE TaxisDB.YellowTaxis
#      CHANGE COLUMN PickupLocationId SET NOT NULL

In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### CHECK constraint

Check constraint helps to enforce certain conditions on the table

In [0]:
spark.sql("""

ALTER TABLE TaxisDB.YellowTaxis

    ADD CONSTRAINT PassengerCountCheck CHECK (PassengerCount <= 5)

""").show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-958133425625982>:1
----> 1 spark.sql("""
      2 
      3 ALTER TABLE TaxisDB.YellowTaxis
      4 
      5     ADD CONSTRAINT PassengerCountCheck CHECK (PassengerCount <= 5)
      6 
      7 """).show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _to_java_column(lit(v)) for k, v in (args or {}).items()}
-> 1387     return DataFrame(self._jsparkSession.

In [0]:
spark.sql("""

ALTER TABLE TaxisDB.YellowTaxis

    ADD CONSTRAINT PassengerCheck CHECK (PassengerCount <= 9 OR PassengerCount IS NULL)

""")

Out[60]: DataFrame[]

In [0]:
spark.sql("""

INSERT INTO TaxisDB.YellowTaxis

--(VendorId, PickupTime, DropTime, PickupLocationId, DropLocationId, PassengerCount, TripDistance, RateCodeId, StoreAndFwdFlag, PaymentType, FareAmount, Extra, MtaTax, TipAmount, TollsAmount, ImprovementSurcharge, TotalAmount, CongestionSurcharge, AirportFee)

VALUES (3, '2022-12-01T00:00:00.000Z', '2022-12-01T00:15:34.000Z', 170, 140, 

        20,  -- PassengerCount

        2.9, 1.0, '1', 1, 13.0, 0.5, 0.5, 1.0, 0.0, 0.3, 15.3, 0.0, 0.0)

""")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-958133425625984>:1
----> 1 spark.sql("""
      2 
      3 INSERT INTO TaxisDB.YellowTaxis
      4 
      5 --(VendorId, PickupTime, DropTime, PickupLocationId, DropLocationId, PassengerCount, TripDistance, RateCodeId, StoreAndFwdFlag, PaymentType, FareAmount, Extra, MtaTax, TipAmount, TollsAmount, ImprovementSurcharge, TotalAmount, CongestionSurcharge, AirportFee)
      6 
      7 VALUES (3, '2022-12-01T00:00:00.000Z', '2022-12-01T00:15:34.000Z', 170, 140, 
      8 
      9         20,  -- PassengerCount
     10 
     11         2.9, 1.0, '1', 1, 13.0, 0.5, 0.5, 1.0, 0.0, 0.3, 15.3, 0.0, 0.0)
     12 
     13 """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
   

In [0]:
spark.sql("""

ALTER TABLE TaxisDB.YellowTaxis

    DROP CONSTRAINT PassengerCheck

""")

Out[62]: DataFrame[]

In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Update a record

In [0]:
# Check passenger count before update

spark.sql("""

SELECT PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show()

+--------------+
|PassengerCount|
+--------------+
|           0.0|
+--------------+



In [0]:
# Update passenger count

spark.sql("""

UPDATE TaxisDB.YellowTaxis

SET PassengerCount = 1

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show()

+-----------------+
|num_affected_rows|
+-----------------+
|                1|
+-----------------+



In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Time Travel: Access using Version Number

In [0]:
# Check at initial version

spark.sql("""

SELECT PassengerCount

FROM TaxisDB.YellowTaxis        VERSION AS OF 0

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show(truncate=False)

+--------------+
|PassengerCount|
+--------------+
+--------------+



In [0]:
# Check at one prior version

spark.sql("""

SELECT PassengerCount

FROM TaxisDB.YellowTaxis        VERSION AS OF 9

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show(truncate=False)

+--------------+
|PassengerCount|
+--------------+
|0.0           |
+--------------+



### Time Travel: Access using Timestamp

In [0]:
spark.sql("""

SELECT PassengerCount

FROM TaxisDB.YellowTaxis        TIMESTAMP AS OF '2024-11-15 22:11:39'

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show(truncate=False)

+--------------+
|PassengerCount|
+--------------+
|0.0           |
+--------------+



### Restore Table to older version

In [0]:
spark.sql("""

RESTORE TABLE TaxisDB.YellowTaxis    TO VERSION AS OF 9

""").show(truncate=False)

+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+
|table_size_after_restore|num_of_files_after_restore|num_removed_files|num_restored_files|removed_files_size|restored_files_size|
+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+
|79999167                |18                        |1                |1                 |5756              |5759               |
+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+



In [0]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+----------------+------------------+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Check data after restore

spark.sql("""

SELECT PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show()

+--------------+
|PassengerCount|
+--------------+
|           0.0|
+--------------+

